# How to write a spelling corrector
Marco Herrero <me@marhs.de> 
(Original idea by [Peter Norvig](http://norvig.com/spell-correct.html))

### Goal
Let's try to write a function called `correct(word)`

```
>>> correct('madriz')
'madrid'
```

In [10]:
def correct(word):
    possible_corrections = []  # ??
    return max(possible_corrections)  # key=??

To define this function, we need to resolve two problems:

1. Given a word, which are the possible corrections of that word?
```
word = 'lates'
possible_corrections = ['late', 'latest']
```

2. Given a set of corrections, which one is the most likely to be the intended?
```
best_correction = late
```

Let's try to use probability to solve both problems

## A little probability theory

Given a word, we are trying to choose the most likely spelling correction for that word (could be the original word!). There is no way to know for sure, so we are going to use probabilites. We are trying to find the correction __c__ that maximizes the probability of __c__ given the original word __w__:

argmax<sub>c</sub> = P(c|w)

By Bayes' Theorem this is equivalent to:

argmax<sub>c</sub> = P(w|c) P(c)<del> / P(w)</del>  

There are three parts of this expression:    
1. _P(c)_: __Language model__: P(c): The probability of a correction stands on its own (the use of the word in a real text)
2. _P(w|c)_: __Error model__: The probability that given a word w, the author meant c.
3. _argmax<sub>x</sub>_: Control mechanism. Enumerate all feasible values of c and choose the one that gives the __best probability score__.

P(w) is useless because is the same for every possible c.

### Language model

Let's say that the probability that a correction __c__ is the valid one is the same that the probability of a word __c__ appears in an text. In and english text:  

__P("the")__ > __P("coriander")__ > __P("xxyxyxxxy")__

To user that, let's make a default dictionary with each word and it's probability given a text. 

In [11]:
import re, collections

def words(text):
    return re.findall('[a-z]+', text.lower())
    
def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

train(words("""Envia una push a todos los usuarios que hayan recibido una push.

                                                             Pushmaster, 2015.
            """))

defaultdict(<function __main__.<lambda>>,
            {'a': 2,
             'envia': 2,
             'hayan': 2,
             'los': 2,
             'push': 3,
             'pushmaster': 2,
             'que': 2,
             'recibido': 2,
             'todos': 2,
             'una': 3,
             'usuarios': 2})

If we use a really big text, like a book, we can get a good __language model__. A language model should represent the language and the context. This is really important. The model can be trained to learn a language inside a context. If I use the historic of my IRC log, the corrector will learn my way of saying thing, accent included. 

In [20]:
filename = 'data/anna_karenina.txt'

In [21]:
NWORDS = train(words(open(filename, 'r').read()))

At this point _NWORDS[**w**]_ holds a count of how many times the word __w__ has been seen in our data. Instead a standard dictionary, we are using a _default dictionary_ with a default value 1. If we want to know the probability of a novel word, we want a default value, less than any word in the text, instead an error. This is called __smoothing__.

### Error model

Now let's look at the problem of enumerating the possible corrections _c_ of a given word _w_. It's common to talk of the __edit distance__ between two words: the number of edits it would take to turn one into the other.

Here is a function that returns a set of all words _c_ that are one edit away from _w_.

In [22]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits_1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]

   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]

   return set(deletes + transposes + replaces + inserts)


This can be a big set. For a word of lenght n, there will be _54n+25_ possible mistakes of lenght one (a few duplicates). Let's not stop here. We want to consider edit distance 2, so let's apply edits_1 to the result of edits_1.

In [23]:
def edits_2(word):
    return set(e2 for e1 in edits_1(word) for e2 in edits_1(e1))

len(edits_2('albaricoque'))

169279

Keep in mind that we have an invalid _word_ that we want to match with a _valid word_, so we need only the valid edits of a word. Let's make a small modification, asking if the edit is in our data.


In [24]:
def known_edits_2(word):
    return set(e2 for e1 in edits_1(word) for e2 in edits_1(e1) if e2 in NWORDS)

known_edits_2('albaricoue')

set()

Now the tricky part. Mistaking one vowel for another is more probable than mistaking two consonants. Making an error on the first leter of a word is less probable, etc. But we have no time, so let's make a shortcut:

All known words of _edit distance_ 1 are more probable than known words of _edit distance_ 2, but less probable than known words of _edit distance_ 0. Let's implement this strategy and the final version of `correct`

In [25]:
def known(words): 
    return set(w for w in words if w in NWORDS)

In [26]:
def correct(word):
    candidates = known([word]) or known(edits_1(word)) or known_edits_2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [32]:
correct('tarenipa')

'karenina'